This notebook is a conceptual research prototype. References to ‘AGI’ or ‘superintelligence’ describe aspirational goals and do not indicate the presence of real general intelligence. Use at your own risk.

# 🌱 *Era-of-Experience* · Colab Notebook
*Alpha-Factory v1 👁️✨ — Lifelong-RL playground*  

A five-minute tour of Sutton & Silver’s **four pillars** — continuous experience, sensor-motor tools, grounded rewards, and non-human reasoning — implemented with the [OpenAI Agents SDK](https://openai.github.io/openai-agents-python/).

### Notebook itinerary
1. ✅ Environment check (GPU / CPU)
2. 📦 Clone repo & install lean deps (≤ 90 s)
3. 🔑 Configure secrets (`OPENAI_API_KEY` optional)
4. 🚀 Launch **Era-of-Experience** agent (Gradio tunnel)
5. 🧪 Probe the agent from Python *(bonus)*

> **Offline-friendly** — leave the key blank and the agent automatically switches to *Mixtral-8x7B-Instruct* via Ollama.

## 0 · Runtime check

In [ ]:
!nvidia-smi -L || echo '🔹 GPU not detected — running on CPU'

## 0b · Verify package versions

In [ ]:
import sys, importlib
def pkg_ver(name):
    try:
        mod = importlib.import_module(name)
        return getattr(mod, '__version__', 'n/a')
    except Exception:
        return 'missing'
print('Python', sys.version.split()[0])
print('openai_agents', pkg_ver('openai_agents'))
print('gradio', pkg_ver('gradio'))

## 1 · Clone repo & install Python deps

In [ ]:
%%bash
set -e
if [ ! -d AGI-Alpha-Agent-v0 ]; then
  echo '📥 Cloning Alpha-Factory v1…'
  git clone --depth 1 https://github.com/MontrealAI/AGI-Alpha-Agent-v0.git
fi
echo '📦 Installing minimal dependencies…'
pip -qq install -U openai_agents gradio aiohttp pretty_errors

## 1b · Verify environment

In [ ]:
!python AGI-Alpha-Agent-v0/check_env.py --auto-install

## 2 · Configure secrets & runtime flags

In [ ]:
import os, getpass, json, textwrap

# 🔑 Supply your OpenAI key or leave blank for offline mode
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI key (blank for offline): ')
os.environ['MODEL_NAME']     = os.getenv('MODEL_NAME', 'gpt-4o-mini')
os.environ['TEMPERATURE']    = os.getenv('TEMPERATURE', '0.4')

print({k:('•••••' if v else '') for k,v in os.environ.items() if k.startswith('OPENAI')})

## 3 · Launch **Era-of-Experience** dashboard

In [ ]:
import subprocess, sys, pathlib, threading, queue, re, time, textwrap

    root = pathlib.Path.cwd() / 'alpha_factory_v1' / 'demos' / 'era_of_experience'
proc = subprocess.Popen([
        sys.executable, 'agent_experience_entrypoint.py'],
        cwd=root, stdout=subprocess.PIPE, stderr=subprocess.STDOUT,
        text=True, bufsize=1)

# -- capture the public Gradio link (Colab tunnels automatically)
link_q = queue.Queue()
def _tail():
    for line in proc.stdout:
        print(line, end='')
        if 'Running on' in line and 'https://' in line:
            m = re.search(r'(https://[\w.-]+\.gradio\.live)', line)
            if m: link_q.put(m.group(1))
threading.Thread(target=_tail, daemon=True).start()

print('⏳ Waiting for Gradio tunnel …')
url = link_q.get()   # blocks until found
print(f'🎉 Open dashboard → {url}')

## 4 · Quick-probe the agent (Python API)

In [ ]:
import asyncio, sys, json, importlib, pathlib
sys.path.append(str(pathlib.Path.cwd() / 'alpha_factory_v1' / 'demos' / 'era_of_experience'))
exp = importlib.import_module('agent_experience_entrypoint')

# an asynchronous generator yields experience events
stream = exp.experience_stream()
async def one_cycle():
    evt   = await anext(stream)
    meal  = await exp.plan_meal(550)
    site  = await exp.web_search('benefits of interval training')
    print(json.dumps({'event': evt, 'meal': meal, 'search': site}, indent=2))

await one_cycle()

## 4b · Available reward backends

In [ ]:
from alpha_factory_v1.demos.era_of_experience import reward_backends
print('Reward modules:', ', '.join(reward_backends.list_rewards()))

## 4c · Sample alpha detection

In [ ]:
import pandas as pd
from pathlib import Path
csv_path = (Path.cwd() / 'alpha_factory_v1' / 'demos' / 'macro_sentinel' / 'offline_samples' / 'yield_curve.csv')
data = pd.read_csv(csv_path)
spread = float(data['10y'][0]) - float(data['3m'][0])
msg = 'Curve inverted → consider long bonds' if spread < 0 else 'Curve normal'
print(f'Yield curve spread: {spread:.2f} | {msg}')


In [ ]:
import pandas as pd
from pathlib import Path
csv_path = (Path.cwd() / 'alpha_factory_v1' / 'demos' / 'macro_sentinel' / 'offline_samples' / 'stable_flows.csv')
data = pd.read_csv(csv_path)
flow = float(data['usd_mn'][0])
msg = 'Bottleneck risk' if flow < 50 else 'Supply chain normal'
print(f'Stable flows: {flow:.1f} M USD | {msg}')


## 5 · Terminate (optional)

## 4d · Run unit tests

In [ ]:
!python -m unittest AGI-Alpha-Agent-v0.tests.test_era_experience -v

In [ ]:
proc.terminate(); proc.wait(); print('✅ Agent stopped')